In [1]:
from flim.experiments import utils, LIDSDataset, ToTensor
from flim.models.lcn import LCNCreator

import torch
from torch import Tensor
from torch import nn
from torch.nn import functional as F
from torch import optim
from torch.utils.data import DataLoader, random_split

from torchvision import transforms

from torchsummary import summary
import torchmetrics as tm

from math import ceil, floor

import copy

from os import path

### Define a CPU/GPU device

In [2]:
if torch.cuda.is_available():
    device = torch.device(0)
else:
    device = 'cpu'

### Define the folder which contains your data

In [3]:
base_dir = path.join("..", "data", "corel")
print(base_dir)

../data/corel


### Train a feature extractor using FLIM

In [4]:
# load architecture
# in this example, the architecure specifies only a feature extractor,
# but it could also be a full model
architecture = utils.load_architecture(path.join(base_dir, 'archs', 'arch-lids.json'))

In [5]:
# load images and markers
images, markers = utils.load_images_and_markers(path.join(base_dir, 'images-and-markers'))
# get input shape
ncols     = images.shape[1]
nrows     = images.shape[2]
nchannels = images.shape[3]
nclasses  = 6

In [6]:
# build model and learn convolutional layers with FLIM
creator = LCNCreator(architecture, images=images, markers=markers, relabel_markers=True, device=device)
creator.build_model()

feature_extractor = creator.get_LIDSConvNet().features

Building m-norm1
Building conv1
Building activation1
Building pool1
Building m-norm2
Building conv2
Building activation2
Building pool2


### Optional: overwrite the weights from another FLIM model

In [7]:
# The model with surrogate weights must have the same architecture and you must inform 
# the folder which contains its parameters

#feature_extractor = utils.load_weights_from_lids_model(feature_extractor, path.join(base_dir, 'param')).to(device)

In [8]:
#feeze the feature extractor to avoid retraining it
for param in feature_extractor.parameters():
    param.requires_grad = False

### Define a MLP Classifier

In [9]:
# switch from train to eval mode
feature_extractor.eval()
# dry run to find out the number of input features for the classifier
ones        = torch.ones(1, nchannels, nrows, ncols, device=device)
in_features = feature_extractor(ones).flatten(start_dim=1).shape[1]

In [10]:
clasifier = nn.Sequential(
    nn.Flatten(1),
    nn.Linear(in_features, 512),
    nn.ReLU(),
    nn.Linear(512, nclasses)
)
# init weights
for layer in clasifier:
    if isinstance(layer, nn.Linear):
        nn.init.xavier_normal_(layer.weight, nn.init.calculate_gain('relu'))
        nn.init.constant_(layer.bias, 0.0)

In [11]:
model = nn.Sequential()
model.add_module("features", feature_extractor)
model.add_module("classifier", clasifier)
model = model.to(device)

In [12]:
summary(model, (nchannels,nrows,ncols))

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 64, 100, 100]        --
|    └─MarkerBasedNorm2d: 2-1            [-1, 3, 400, 400]         (6)
|    └─Conv2d: 2-2                       [-1, 32, 400, 400]        (2,400)
|    └─ReLU: 2-3                         [-1, 32, 400, 400]        --
|    └─MaxPool2d: 2-4                    [-1, 32, 200, 200]        --
|    └─MarkerBasedNorm2d: 2-5            [-1, 32, 200, 200]        (64)
|    └─Conv2d: 2-6                       [-1, 64, 200, 200]        (18,432)
|    └─ReLU: 2-7                         [-1, 64, 200, 200]        --
|    └─MaxPool2d: 2-8                    [-1, 64, 100, 100]        --
├─Sequential: 1-2                        [-1, 6]                   --
|    └─Flatten: 2-9                      [-1, 640000]              --
|    └─Linear: 2-10                      [-1, 512]                 327,680,512
|    └─ReLU: 2-11                        [-1, 512]            

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 64, 100, 100]        --
|    └─MarkerBasedNorm2d: 2-1            [-1, 3, 400, 400]         (6)
|    └─Conv2d: 2-2                       [-1, 32, 400, 400]        (2,400)
|    └─ReLU: 2-3                         [-1, 32, 400, 400]        --
|    └─MaxPool2d: 2-4                    [-1, 32, 200, 200]        --
|    └─MarkerBasedNorm2d: 2-5            [-1, 32, 200, 200]        (64)
|    └─Conv2d: 2-6                       [-1, 64, 200, 200]        (18,432)
|    └─ReLU: 2-7                         [-1, 64, 200, 200]        --
|    └─MaxPool2d: 2-8                    [-1, 64, 100, 100]        --
├─Sequential: 1-2                        [-1, 6]                   --
|    └─Flatten: 2-9                      [-1, 640000]              --
|    └─Linear: 2-10                      [-1, 512]                 327,680,512
|    └─ReLU: 2-11                        [-1, 512]            

### Train your classifier

In [13]:
def get_metrics() -> dict:
    """
    Returns a dictionary of metrics to compute.
    """
    return {
        "acc": tm.Accuracy().to(device),
        "kappa": tm.CohenKappa(num_classes=nclasses).to(device),
    }

def training_step(model: nn.Module,
                  batch_x: Tensor,
                  batch_y: Tensor,
                  optimizer: optim.Optimizer,
                  metrics: dict) -> Tensor:
    """
    One training step over one batch (batch_x, batch_y).

    Parameters
    ----------
    model   : nn.Module
        Model to train.
    batch_x : torch.Tensor
        Input batch.
    batch_y : torch.Tensor
        Target batch.
    optimizer : optim.Optimizer
        Optimizer to use.
    metrics : dict
        Metrics to compute.
    Returns
    -------
    loss : Tensor
        Loss computed on the batch.
    """
    optimizer.zero_grad()
    output = model(batch_x)
    loss = F.cross_entropy(output, batch_y)
    nn.utils.clip_grad_norm_(model.parameters(), 0.1)
    loss.backward()
    optimizer.step()
    

    for _, metric in metrics.items():
        metric(output.softmax(dim=1), batch_y)
    
    return loss

def evaluation_step(model: nn.Module, 
                    batch_x: torch.Tensor,
                    batch_y: torch.Tensor,
                    metrics: dict) -> Tensor:
    """
    One evaluation step over one batch (batch_x, batch_y).

    Parameters
    ----------
    model   : nn.Module
        Model to evaluate.
    batch_x : torch.Tensor
        Input batch.
    batch_y : torch.Tensor
        Target batch.
    metrics : dict
        Metrics to compute.
    Returns
    -------
    loss: Tensor
        Loss computed on the batch.
    """
    output = model(batch_x)
    loss = F.cross_entropy(output, batch_y)

    for _, metric in metrics.items():
        metric(output.softmax(dim=1), batch_y)
    
    return loss

In [14]:
# Define train set. It should be a folder with images named as 0000{label}_0000{imge_number}.{image_format}
# You can create a .txt file with the name of the images in the train set and another file with the name of the images in test set
trainset = LIDSDataset(path.join(base_dir, 'dataset'), path.join(base_dir, 'train.txt'), transform=ToTensor())
# split train set in train and validation set
# consider forcing splits to be stratified
trainset, valset = random_split(trainset, [ceil(len(trainset)*0.8), floor(len(trainset)*0.2)])

In [15]:
trainloader = DataLoader(trainset, batch_size=32, shuffle=True, drop_last=False)
valloader   = DataLoader(valset, batch_size=32, shuffle=False, drop_last=False)

In [16]:
lr      = 0.00001
w_decay = 0.1
epochs  = 30

# set feature extraction in evaluation mode  
model.features.eval()

# get the trainable parameters
params = [param for param in model.parameters() if param.requires_grad]

# optimizer
optimizer = optim.Adam(params, lr=lr, weight_decay=w_decay)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, cooldown=3, factor=0.1, verbose=False)
# get evaluation metrics      
train_metrics = get_metrics()
val_metrics   = get_metrics()

/home/afalcao/miniconda3/envs/FLIM/lib/python3.8/site-packages/pytorch_lightning/metrics/__init__.py:43: LightningDeprecationWarning: `pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package (https://github.com/PyTorchLightning/metrics) since v1.3 and will be removed in v1.5
  rank_zero_deprecation(


In [17]:
# save the model with the best validation kappa
best_model = None
best_kappa = -1

# training loop
for epoch in range(0, epochs):
    # training setp
    N = len(trainloader)
    train_loss = 0 
    for i, data in enumerate(trainloader):
        model.classifier.train()
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        loss = training_step(model, inputs, labels, optimizer, train_metrics)
        train_loss += loss
        print(f"\r[training]\tepoch: {epoch}/{epochs-1}\titeration: {i}/{N-1}\tloss: {loss.item():.6f}", end="")

    train_loss = train_loss / N
    train_acc = train_metrics['acc'].compute()
    train_kappa = train_metrics['kappa'].compute()

    print(f"\r[training]\tepoch: {epoch}/{epochs-1}\tloss: {train_loss.item():.6f}\tacc: {train_acc:.6f}\tkappa: {train_kappa:.6f}")

    # validation step
    N = len(valloader)
    val_loss = 0
    for i, data in enumerate(valloader):
        with torch.no_grad():
            model.classifier.eval()
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            loss = evaluation_step(model, inputs, labels, val_metrics)
            val_loss += loss
            print(f"\r[validating]\tepoch: {epoch}/{epochs-1}\titeration: {i}/{N-1}\tloss: {loss.item():.6f}", end="")

    val_loss = val_loss / N
    val_acc = val_metrics["acc"].compute()
    val_kappa = val_metrics["kappa"].compute()

    print(f"\r[validating]\tepoch: {epoch}/{epochs-1}\tloss: {val_loss.item():.6f}\tacc: {val_acc:.6f}\tkappa: {val_kappa:.6f}")

    # save the model if it has the best validation kappa
    if epoch == 0 or val_kappa > best_kappa:
        best_kappa = val_kappa
        best_model = {k: v.cpu() for k, v in model.state_dict().items()}

    scheduler.step(val_loss)

[training]	epoch: 0/29	loss: 2.512415	acc: 0.366071	kappa: 0.212127
[validating]	epoch: 0/29	loss: 2.183391	acc: 0.481481	kappa: 0.343750
[training]	epoch: 1/29	loss: 0.753515	acc: 0.549107	kappa: 0.446196
[validating]	epoch: 1/29	loss: 1.504442	acc: 0.555556	kappa: 0.440414
[training]	epoch: 2/29	loss: 0.701454	acc: 0.636905	kappa: 0.553260
[validating]	epoch: 2/29	loss: 0.905116	acc: 0.592593	kappa: 0.484673
[training]	epoch: 3/29	loss: 0.275817	acc: 0.703125	kappa: 0.634687
[validating]	epoch: 3/29	loss: 0.969730	acc: 0.601852	kappa: 0.500000
[training]	epoch: 4/29	loss: 0.106598	acc: 0.753571	kappa: 0.696911
[validating]	epoch: 4/29	loss: 0.680701	acc: 0.637037	kappa: 0.539185
[training]	epoch: 5/29	loss: 0.089724	acc: 0.788690	kappa: 0.740059
[validating]	epoch: 5/29	loss: 0.623013	acc: 0.672840	kappa: 0.582555
[training]	epoch: 6/29	loss: 0.022066	acc: 0.818878	kappa: 0.777252
[validating]	epoch: 6/29	loss: 0.630299	acc: 0.703704	kappa: 0.621053
[training]	epoch: 7/29	loss: 0.031

### Evaluate your model

In [18]:
# Define test set. It should be a folder with images named as 0000{label}_0000{imge_number}.{image_format}
# You can create a .txt file with the name of the images in the test set
testset    = LIDSDataset(path.join(base_dir, 'dataset'), path.join(base_dir, 'test.txt'), transform=ToTensor())
testloader = DataLoader(testset, batch_size=32, shuffle=False, drop_last=False)

In [19]:
# get evalutation metrics
test_metrics = get_metrics()

# set model in evaluation mode
model.load_state_dict(best_model, )
model.eval()

# testing loop
N = len(testloader)
with torch.no_grad():
    for i, data in enumerate(testloader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        loss = evaluation_step(model, inputs, labels, test_metrics)
        print(f"\r[testing]\titeration: {i}/{N-1}\tloss: {loss.item()}", end="")

test_acc = test_metrics["acc"].compute()
test_kappa = test_metrics["kappa"].compute()

print(f"\r[testing] acc: {test_acc:.6f}\tkappa: {test_kappa:.6f}")

[testing] acc: 0.921296	kappa: 0.90372894391822815


### Train and evaluate a SVM classifier using your feature extractor

In [20]:
svm = utils.train_svm(feature_extractor, trainset, batch_size=32, max_iter=10000, C=1e2, device=device)

Preparing to train SVM
Fitting SVM...
Done


In [21]:
utils.validate_svm(feature_extractor, svm, testset, batch_size=32, device=device)

Calculating metrics...
##################################################
Acc : 0.912037
--------------------------------------------------
F1-score : 0.912215
--------------------------------------------------
Accuracy 0.9074074074074074 0.8666666666666667 1.0 0.95 0.896551724137931 0.9024390243902439
--------------------------------------------------
Precision: 0.9607843137254902 0.9285714285714286 0.9310344827586207 0.9047619047619048 0.8125 0.9024390243902439
Recall: 0.9074074074074074 0.8666666666666667 1.0 0.95 0.896551724137931 0.9024390243902439
F-score: 0.9333333333333333 0.896551724137931 0.9642857142857143 0.9268292682926829 0.8524590163934426 0.9024390243902439
--------------------------------------------------
W-Precision: 0.9141839646139426
W-Recall: 0.9120370370370371
W-F-score: 0.9122149940425216
--------------------------------------------------
Kappa 0.892565445026178
--------------------------------------------------
Suport 54 45 27 20 29 41
#########################